## Imports

In [2]:
import trimesh
import open3d as o3d
import numpy as np
from sklearn.cluster import KMeans
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from open3d.visualization import gui
from open3d.visualization import rendering
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

C:\Users\Arif\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Load and compute the mean curvature

In [3]:
# Load the tooth STL model using Trimesh
mesh_trimesh = trimesh.load_mesh("stlFiles/sinifBirNumaraAlti.stl")

# Get vertices, faces, and normals
vertices = np.array(mesh_trimesh.vertices)
faces = np.array(mesh_trimesh.faces)
normals = np.array(mesh_trimesh.vertex_normals)
tooth_o3d = o3d.geometry.TriangleMesh()
tooth_o3d.vertices = o3d.utility.Vector3dVector(vertices)
tooth_o3d.triangles = o3d.utility.Vector3iVector(faces)
tooth_o3d.compute_vertex_normals()# Convert full tooth to Open3D mesh

tooth_o3d.paint_uniform_color([0.8, 0.8, 0.8])  # light gray


# Compute Mean Curvature using Trimesh
mean_curvature = trimesh.curvature.discrete_mean_curvature_measure(mesh_trimesh, mesh_trimesh.vertices, radius=2)
print('Discrete Mean curvature calculated')


Discrete Mean curvature calculated


### The function that extracts the cavity from the tooth model

In [4]:
def extract_largest_cavity(vertices, faces, cavity_indices):
    # Get unique cavity indices
    unique_cavity_indices = np.unique(cavity_indices)
    
    # Find faces that have all vertices in cavity_indices
    cavity_face_mask = np.isin(faces.ravel(), unique_cavity_indices).reshape(faces.shape)
    cavity_face_indices = np.where(np.all(cavity_face_mask, axis=1))[0]
    cavity_faces = faces[cavity_face_indices]
    
    # Create adjacency matrix for connected component analysis
    edges = set()
    for face in cavity_faces:
        edges.add((face[0], face[1]))
        edges.add((face[1], face[2]))
        edges.add((face[2], face[0]))
    
    # Create sparse adjacency matrix
    row, col = zip(*edges)
    row = np.array(row)
    col = np.array(col)
    data = np.ones_like(row)
    
    # Create sparse matrix with size equal to total vertices 
    # (will be pruned to cavity vertices later)
    adj_matrix = csr_matrix((data, (row, col)), shape=(vertices.shape[0], vertices.shape[0]))
    
    # Find connected components
    n_components, labels = connected_components(csgraph=adj_matrix, directed=False)
    
    # Count vertices in each component
    component_sizes = np.zeros(n_components, dtype=int)
    for label in labels[unique_cavity_indices]:
        component_sizes[label] += 1
    
    # Find the largest component
    largest_component = np.argmax(component_sizes)
    
    # Get vertices from the largest component
    largest_cavity_indices = np.where(labels == largest_component)[0]
    largest_cavity_indices = np.intersect1d(largest_cavity_indices, unique_cavity_indices)
    
    # Create index mapping for new mesh
    index_map = np.zeros(len(vertices), dtype=int)
    for i, idx in enumerate(largest_cavity_indices):
        index_map[idx] = i
    
    # Get faces for largest component
    largest_face_mask = np.isin(cavity_faces.ravel(), largest_cavity_indices).reshape(cavity_faces.shape)
    largest_face_indices = np.where(np.all(largest_face_mask, axis=1))[0]
    largest_cavity_faces = cavity_faces[largest_face_indices]
    
    # Remap face indices
    remapped_faces = np.zeros_like(largest_cavity_faces)
    for i in range(largest_cavity_faces.shape[0]):
        for j in range(3):
            remapped_faces[i, j] = index_map[largest_cavity_faces[i, j]]
    
    # Create and return the largest cavity mesh
    largest_cavity_mesh = o3d.geometry.TriangleMesh()
    largest_cavity_mesh.vertices = o3d.utility.Vector3dVector(vertices[largest_cavity_indices])
    largest_cavity_mesh.triangles = o3d.utility.Vector3iVector(remapped_faces)
    largest_cavity_mesh.compute_vertex_normals()
    
    # Set color for visualization
    cavity_colors = np.ones((len(largest_cavity_indices), 3)) * [0, 1, 0]  # Green
    largest_cavity_mesh.vertex_colors = o3d.utility.Vector3dVector(cavity_colors)
    
    return largest_cavity_mesh, largest_cavity_indices

### The function that extracts the cavity bottom 

In [5]:

def extract_cavity_parts(largest_cavity_mesh, bottom_threshold_percentage=0.1):
    # Get vertices and triangles from the mesh
    cavity_vertices = np.asarray(largest_cavity_mesh.vertices)
    cavity_triangles = np.asarray(largest_cavity_mesh.triangles)
    
    # Calculate z-range
    min_z = np.min(cavity_vertices[:, 2])
    max_z = np.max(cavity_vertices[:, 2])
    z_range = max_z - min_z
    
    # Define a threshold for what constitutes the "bottom"
    z_threshold = min_z + z_range * bottom_threshold_percentage
    
    # Find vertices that are in the bottom region
    bottom_vertex_mask = cavity_vertices[:, 2] <= z_threshold
    bottom_vertex_indices = np.where(bottom_vertex_mask)[0]
    
    # Find triangles where all three vertices are in the bottom region
    bottom_triangles_mask = np.isin(cavity_triangles.ravel(), bottom_vertex_indices).reshape(cavity_triangles.shape)
    bottom_triangle_indices = np.where(np.all(bottom_triangles_mask, axis=1))[0]
    
    # Find triangles that are NOT in the bottom (side triangles)
    total_triangles = len(cavity_triangles)
    side_triangle_indices = np.setdiff1d(np.arange(total_triangles), bottom_triangle_indices)
    
    result = {}
    
    # Process bottom mesh
    if len(bottom_triangle_indices) > 0:
        # Create a new mesh for the bottom surface
        bottom_triangles = cavity_triangles[bottom_triangle_indices]
        
        # Get unique vertices used in the bottom triangles
        unique_vertices = np.unique(bottom_triangles.ravel())
        
        # Create index mapping
        index_map = np.zeros(len(cavity_vertices), dtype=int)
        for i, idx in enumerate(unique_vertices):
            index_map[idx] = i
        
        # Remap triangle indices
        remapped_triangles = np.zeros_like(bottom_triangles)
        for i in range(bottom_triangles.shape[0]):
            for j in range(3):
                remapped_triangles[i, j] = index_map[bottom_triangles[i, j]]
        
        # Create bottom surface mesh
        bottom_mesh = o3d.geometry.TriangleMesh()
        bottom_mesh.vertices = o3d.utility.Vector3dVector(cavity_vertices[unique_vertices])
        bottom_mesh.triangles = o3d.utility.Vector3iVector(remapped_triangles)
        bottom_mesh.compute_vertex_normals()
        
        # Set color for visualization
        bottom_colors = np.ones((len(unique_vertices), 3)) * [0, 0, 1]  # Blue for bottom surface
        bottom_mesh.vertex_colors = o3d.utility.Vector3dVector(bottom_colors)
        
    else:
        print("No triangles found in the bottom region. Try adjusting the threshold.")
    
    # Process side mesh
    if len(side_triangle_indices) > 0:
        # Create a new mesh for the side surface
        side_triangles = cavity_triangles[side_triangle_indices]
        
        # Get unique vertices used in the side triangles
        unique_vertices = np.unique(side_triangles.ravel())
        
        # Create index mapping
        index_map = np.zeros(len(cavity_vertices), dtype=int)
        for i, idx in enumerate(unique_vertices):
            index_map[idx] = i
        
        # Remap triangle indices
        remapped_triangles = np.zeros_like(side_triangles)
        for i in range(side_triangles.shape[0]):
            for j in range(3):
                remapped_triangles[i, j] = index_map[side_triangles[i, j]]
        
        # Create side surface mesh
        side_mesh = o3d.geometry.TriangleMesh()
        side_mesh.vertices = o3d.utility.Vector3dVector(cavity_vertices[unique_vertices])
        side_mesh.triangles = o3d.utility.Vector3iVector(remapped_triangles)
        side_mesh.compute_vertex_normals()
        
        # Set color for visualization
        side_colors = np.ones((len(unique_vertices), 3)) * [1, 0, 0]  # Red for side surface
        side_mesh.vertex_colors = o3d.utility.Vector3dVector(side_colors)
        
    else:
        print("No triangles found for the side region.")
    
    return side_mesh , bottom_mesh

### The function that calulates the cavity_bottom roughness

In [6]:
def calculate_roughness(mesh):
    mesh_vertices = np.asarray(mesh.vertices)
    mesh_faces = np.asarray(mesh.triangles)
    tri_mesh = trimesh.Trimesh(vertices=mesh_vertices, faces=mesh_faces, process=False)
    normals = tri_mesh.face_normals
    adj = tri_mesh.face_adjacency

    # Compute angle between adjacent face normals
    dot = np.einsum('ij,ij->i', normals[adj[:, 0]], normals[adj[:, 1]])
    angles = np.arccos(np.clip(dot, -1.0, 1.0)) # Mean angle
    angles_deg = np.degrees(angles) # roughness
    return angles_deg


### kavitenin seçilmesi 

In [ ]:
cavity_indices = np.where(mean_curvature < 0.4)[0]  # Select all vertices with negative curvature
mesh, largest_cavity_indices = extract_largest_cavity(vertices, faces, cavity_indices)
cavity_vertices= np.asarray(mesh.vertices)


In [ ]:
def extract_top_percentage(cavity_mesh, percentage=28.0):
    """
    Z ekseninde yukarıdan belirtilen yüzde kadarını alarak yeni bir mesh döndürür.
    
    Args:
        cavity_mesh: İşlenecek Open3D mesh
        percentage: Yukarıdan alınacak yüzde (varsayılan %1)
        
    Returns:
        Yukarıdan belirtilen yüzde kadar kesilmiş yeni bir O3D mesh
    """
    # Mesh'i kopyala (orijinal mesh'i değiştirmemek için)
    clipped_mesh = o3d.geometry.TriangleMesh(cavity_mesh)
    
    # Vertexleri numpy array olarak al
    vertices = np.asarray(clipped_mesh.vertices)
    triangles = np.asarray(clipped_mesh.triangles)
    
    if len(vertices) == 0:
        print("Mesh içinde vertex bulunamadı!")
        return None
    
    # Z eksenindeki min ve max değerleri bul
    min_z = np.min(vertices[:, 2])
    max_z = np.max(vertices[:, 2])
    
    # Z yüksekliğini hesapla
    height = max_z - min_z
    
    # Z ekseni boyunca kesim noktasını belirle (yukarıdan %1'lik dilim için)
    z_threshold = max_z - (height * percentage / 100.0)
    
    # Sadece z_threshold'dan büyük z değerlerine sahip vertexleri seç
    mask = vertices[:, 2] >= z_threshold
    
    if np.sum(mask) == 0:
        print(f"Belirtilen yüzde (%{percentage}) için mesh üzerinde vertex bulunamadı!")
        return None
    
    # Yeni mesh oluştur
    result_mesh = o3d.geometry.TriangleMesh()
    
    # Yeni mesh için filtreleme yaklaşımı:
    # 1. Hangi üçgenlerin tamamen eşik değerinin üzerinde olduğunu belirle
    valid_triangles = []
    for triangle in triangles:
        v1, v2, v3 = triangle
        # Üçgenin tüm köşeleri eşik değerinden yukarıdaysa, bu üçgeni dahil et
        if vertices[v1, 2] >= z_threshold and vertices[v2, 2] >= z_threshold and vertices[v3, 2] >= z_threshold:
            valid_triangles.append(triangle)
    
    if len(valid_triangles) == 0:
        print(f"Belirtilen yüzde (%{percentage}) için geçerli üçgen bulunamadı!")
        return None
    
    # Eşik üzerindeki vertexlerin indekslerini ve yeni indeks haritalamasını oluştur
    valid_vertices_indices = np.where(mask)[0]
    index_map = {old_idx: new_idx for new_idx, old_idx in enumerate(valid_vertices_indices)}
    
    # Yeni vertexleri ekle
    new_vertices = vertices[mask]
    result_mesh.vertices = o3d.utility.Vector3dVector(new_vertices)
    
    # Yeni üçgenleri ekle (indeks haritalamasını kullanarak)
    new_triangles = []
    for triangle in valid_triangles:
        try:
            new_triangle = [index_map[idx] for idx in triangle]
            new_triangles.append(new_triangle)
        except KeyError:
            # Eğer üçgen tamamen eşik üzerinde değilse, KeyError olabilir
            continue
    
    result_mesh.triangles = o3d.utility.Vector3iVector(new_triangles)
    
    # Vertex normallerini yeniden hesapla
    result_mesh.compute_vertex_normals()
    
    print(f"Orijinal mesh: {len(vertices)} vertex, {len(triangles)} üçgen")
    print(f"Kırpılmış mesh: {len(new_vertices)} vertex, {len(new_triangles)} üçgen")
    print(f"Z ekseni aralığı: {z_threshold:.4f} - {max_z:.4f} (orijinal: {min_z:.4f} - {max_z:.4f})")
    
    return result_mesh
top_tooth_mesh = extract_top_percentage(tooth_o3d) 
# o3d.visualization.draw_geometries([top_tooth_mesh]) 


Orijinal mesh: 16381 vertex, 32626 üçgen
Kırpılmış mesh: 5536 vertex, 10599 üçgen
Z ekseni aralığı: 2.4606 - 4.7851 (orijinal: -3.5166 - 4.7851)


### Experimental : Get height and width of the largest_cavity

In [28]:
def get_mesh_length_height_lines(top_mesh):
    """
    Z ekseninde yukarıdan belirtilen yüzde kadarını alarak yeni bir mesh oluşturur,
    OBB kullanarak boyutlarını hesaplar ve bu boyutları temsil eden çizgileri döndürür.
    
    Args:
        cavity_mesh: İşlenecek Open3D mesh
        percentage: Yukarıdan alınacak yüzde (varsayılan %1)
        
    Returns:
        (top_mesh, line_set, obb): Üst kısım mesh'i, boyut çizgileri ve OBB
    """
    # Üst kısmı al
    
    # Mesh'i OBB (Oriented Bounding Box) ile analiz et
    obb = top_mesh.get_oriented_bounding_box()
    
    # OBB'nin boyutlarını ve yönelimini al
    extent = obb.extent
    rotation = obb.R  # OBB'nin dönüş matrisi
    center = obb.center  # OBB'nin merkezi
    
    # Boyutları ve karşılık gelen eksenleri belirle
    dimensions_with_axes = [(extent[i], rotation[:, i]) for i in range(3)]
    # Boyutlara göre sırala (büyükten küçüğe)
    dimensions_with_axes.sort(key=lambda x: x[0], reverse=True)
    
    # En büyük iki boyut genişlik ve uzunluk olarak kabul edilir
    width, width_axis = dimensions_with_axes[0]
    length, length_axis = dimensions_with_axes[1]
    height, height_axis = dimensions_with_axes[2]
    
    # Boyut çizgilerini oluştur
    line_points = []
    line_indices = []
    line_colors = []
    
    # Merkez noktadan başlayan çizgiler oluştur
    
    # Genişlik çizgisi (kırmızı)
    line_points.append(center)  # Başlangıç noktası (merkez)
    line_points.append(center + width_axis * width/2)  # Bitiş noktası
    line_indices.append([0, 1])  # İlk çizgi: 0. ve 1. noktalar arasında
    line_colors.append([1, 0, 0])  # Kırmızı
    
    # Uzunluk çizgisi (yeşil)
    line_points.append(center)  # Başlangıç noktası (merkez)
    line_points.append(center + length_axis * length/2)  # Bitiş noktası
    line_indices.append([0, 3])  # İkinci çizgi: 0. ve 3. noktalar arasında
    line_colors.append([0, 1, 0])  # Yeşil
    
    # Yükseklik çizgisi (mavi)
    line_points.append(center)  # Başlangıç noktası (merkez)
    line_points.append(center + height_axis * height/2)  # Bitiş noktası
    line_indices.append([0, 5])  # Üçüncü çizgi: 0. ve 5. noktalar arasında
    line_colors.append([0, 0, 1])  # Mavi
    
    # Negatif yönde de çizgi ekle (çift taraflı)
    
    # Negatif genişlik
    line_points.append(center - width_axis * width/2)
    line_indices.append([0, 6])
    line_colors.append([1, 0, 0])  # Kırmızı
    
    # Negatif uzunluk
    line_points.append(center - length_axis * length/2)
    line_indices.append([0, 7])
    line_colors.append([0, 1, 0])  # Yeşil
    
    # Negatif yükseklik
    line_points.append(center - height_axis * height/2)
    line_indices.append([0, 8])
    line_colors.append([0, 0, 1])  # Mavi
    
    # LineSet oluştur
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(line_points)
    line_set.lines = o3d.utility.Vector2iVector(line_indices)
    line_set.colors = o3d.utility.Vector3dVector(line_colors)
    
    print(f"OBB boyutları: Genişlik={width:.4f}, Uzunluk={length:.4f}, Yükseklik={height:.4f}")
    
    # OBB'yi turuncu renkle göster
    obb.color = np.array([1, 0.5, 0])
    
    return line_set, obb, width, length, height
line_set, obb, width, length, height = get_mesh_length_height_lines(top_tooth_mesh)

o3d.visualization.draw_geometries([top_tooth_mesh, line_set, obb])

OBB boyutları: Genişlik=11.7847, Uzunluk=10.0157, Yükseklik=2.5244


### Kavitenin alt kısmının seçilmesi ve kavite yüksekliğinin hesaplanması

In [ ]:
side_bottom, cavity_bottom = extract_cavity_parts(mesh, bottom_threshold_percentage=0.4)

# kavite altının z eksenindeki ortalamasını al
bottom_vertices = np.asarray(cavity_bottom.vertices)
bottom_z_values = bottom_vertices[:, 2]
min_z_mean = np.mean(bottom_z_values) 

# kavite alanının en üstü 
max_z = np.max(cavity_vertices[:, 2])
cavity_depth = max_z - min_z_mean  # Derinlik (Z eksenindeki fark)



### Roughness Görselleştirme

In [52]:
def visualize_roughness(cavity_bottom, full_tooth_mesh):
    # Cavity'den hesaplama yap
    cavity_vertices = np.asarray(cavity_bottom.vertices)
    z_values = cavity_vertices[:, 2]
    z_mean = np.mean(z_values)
    
    # Cavity için sapmaları hesapla
    deviations = np.abs(z_values - z_mean)
    max_deviation = np.max(deviations)
    
    # Cavity için normalize edilmiş sapmaları hesapla
    cavity_normalized_deviations = deviations / max_deviation if max_deviation > 0 else np.zeros_like(deviations)
    
    # Tüm diş modeli için renk arrayi oluştur
    full_vertices = np.asarray(full_tooth_mesh.vertices)
    colors = np.zeros((len(full_vertices), 3))
    
    # Önce tüm dişi mavi yap (düz alanlar)
    colors[:, :] = [0.5, 0.5, 0.5]  # Medium gray
    
    # Cavity vertex'lerinin tam diş modelindeki indekslerini bul ve bu pozisyonları renklendir
    # NOT: Bu kısım, cavity_bottom vertex'lerinin full_tooth_mesh içindeki aynı koordinatlara sahip vertex'leri bulma mantığına dayanır
    
    # Basit yaklaşım: Her cavity vertex'i için en yakın full_tooth vertex'ini bul
    from scipy.spatial import KDTree
    
    # KDTree oluştur tüm diş vertexleri için
    tree = KDTree(full_vertices)
    
    # Her cavity vertex'i için en yakın diş vertex'ini bul
    _, indices = tree.query(cavity_vertices)
    
    # Cavity vertex'lerinin renklerini tüm diş modelindeki karşılık gelen vertex'lere uygula
    colors[indices, 0] = cavity_normalized_deviations  # Kırmızı kanal
    colors[indices, 2] = 1.0 - cavity_normalized_deviations  # Mavi kanal
    
    # Renkleri tüm diş mesh'ine uygula
    colored_roughness_mesh = o3d.geometry.TriangleMesh()
    colored_roughness_mesh.vertices = full_tooth_mesh.vertices
    colored_roughness_mesh.triangles = full_tooth_mesh.triangles
    colored_roughness_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)
    colored_roughness_mesh.compute_vertex_normals()
    
    return colored_roughness_mesh

### Roughness Hesaplanması

In [66]:

outline_indices = np.where((mean_curvature > 3.0))[0]
roughness = calculate_roughness(cavity_bottom)
colored_roughness = visualize_roughness(cavity_bottom,tooth_o3d)
print("Mean angle between adjacent faces:", np.mean(roughness))
print("Standard deviation (roughness):", np.std(roughness))


Mean angle between adjacent faces: 6.076013062789825
Standard deviation (roughness): 4.21311751638555


### Görüntüleme

In [67]:
# **Çizgiyi tanımlama**
cavity_centroid = np.mean(cavity_vertices, axis=0)
min_z_point = [cavity_centroid[0], cavity_centroid[1], min_z_mean]
max_z_point = [cavity_centroid[0], cavity_centroid[1], max_z]
line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector([min_z_point, max_z_point])
line_set.lines = o3d.utility.Vector2iVector([[0, 1]])
line_set.colors = o3d.utility.Vector3dVector([[1, 0, 0]])  # Mavi çizgi

print("cavity_depth : ",cavity_depth)
o3d.io.write_triangle_mesh("output/colored_roughness.ply", colored_roughness , write_vertex_colors=True)

#colored_roughness, cavity_bottom, line_set, largest_cavity_mesh, tooth_o3d
o3d.visualization.draw_geometries([colored_roughness,],mesh_show_back_face=True)

cavity_depth :  2.0301277332356844


In [60]:
def split_side_and_get_normal_means(side_mesh):
    """
    Splits a side mesh into right and left walls and returns their mean normal vectors.
    
    Args:
        side_mesh: Open3D triangle mesh representing side walls
        
    Returns:
        right_mesh: The right side wall mesh
        left_mesh: The left side wall mesh
        right_normal_mean: Mean normal vector of the right mesh
        left_normal_mean: Mean normal vector of the left mesh
    """
    # Ensure we have triangle normals
    side_mesh.compute_triangle_normals()
    triangle_normals = np.asarray(side_mesh.triangle_normals)
    
    # Project normals onto the x-y plane
    xy_normals = triangle_normals[:, :2]
    
    # Calculate angles in the x-y plane (in radians)
    angles = np.arctan2(xy_normals[:, 1], xy_normals[:, 0])
    
    # Convert to degrees and shift to 0-360 range
    angles_deg = np.degrees(angles) % 360
    
    # Define right (0-180) and left (180-360) indices
    right_indices = np.where((angles_deg >= 0) & (angles_deg < 180))[0]
    left_indices = np.where((angles_deg >= 180) & (angles_deg < 360))[0]
    
    # Calculate mean normal for right side
    right_normals = triangle_normals[right_indices]
    right_normal_mean = np.mean(right_normals, axis=0)
    # Normalize the mean normal vector
    right_normal_mean = right_normal_mean / np.linalg.norm(right_normal_mean)
    
    # Calculate mean normal for left side
    left_normals = triangle_normals[left_indices]
    left_normal_mean = np.mean(left_normals, axis=0)
    # Normalize the mean normal vector
    left_normal_mean = left_normal_mean / np.linalg.norm(left_normal_mean)
    
    # Create the right mesh
    vertices = np.asarray(side_mesh.vertices)
    triangles = np.asarray(side_mesh.triangles)
    
    right_triangles = triangles[right_indices]
    unique_right_vertices = np.unique(right_triangles.ravel())
    
    vertex_map_right = np.zeros(len(vertices), dtype=int)
    for i, idx in enumerate(unique_right_vertices):
        vertex_map_right[idx] = i
        
    remapped_right_triangles = np.zeros_like(right_triangles)
    for i in range(right_triangles.shape[0]):
        for j in range(3):
            remapped_right_triangles[i, j] = vertex_map_right[right_triangles[i, j]]
            
    right_mesh = o3d.geometry.TriangleMesh()
    right_mesh.vertices = o3d.utility.Vector3dVector(vertices[unique_right_vertices])
    right_mesh.triangles = o3d.utility.Vector3iVector(remapped_right_triangles)
    
    # Create the left mesh
    left_triangles = triangles[left_indices]
    unique_left_vertices = np.unique(left_triangles.ravel())
    
    vertex_map_left = np.zeros(len(vertices), dtype=int)
    for i, idx in enumerate(unique_left_vertices):
        vertex_map_left[idx] = i
        
    remapped_left_triangles = np.zeros_like(left_triangles)
    for i in range(left_triangles.shape[0]):
        for j in range(3):
            remapped_left_triangles[i, j] = vertex_map_left[left_triangles[i, j]]
            
    left_mesh = o3d.geometry.TriangleMesh()
    left_mesh.vertices = o3d.utility.Vector3dVector(vertices[unique_left_vertices])
    left_mesh.triangles = o3d.utility.Vector3iVector(remapped_left_triangles)
    
    return right_mesh, left_mesh, right_normal_mean, left_normal_mean

In [61]:
right_mesh, left_mesh, right_normal_mean, left_normal_mean = split_side_and_get_normal_means(side_bottom)

print("right_normal_mean : " ,right_normal_mean)
print("left_normal_mean : " ,left_normal_mean)
cavity_bottom.compute_vertex_normals()
bottom_normal_mean = np.mean(np.asarray(cavity_bottom.vertex_normals),axis=0)

right_angle = np.dot(right_normal_mean, bottom_normal_mean)
left_angle = np.dot(left_normal_mean, bottom_normal_mean)

print("right_angle : " ,right_angle)
print("left_angle : " ,left_angle)


o3d.visualization.draw_geometries([right_mesh])


right_normal_mean :  [-0.11308598  0.98379338  0.1391479 ]
left_normal_mean :  [-0.05005476 -0.98579777  0.16030369]
right_angle :  0.08822044773719756
left_angle :  0.11858101775956988


In [64]:
import json
data = {
    "right_angle": right_angle,
    "left_angle": left_angle,
    "cavity_depth": cavity_depth,
    "roughness": np.std(roughness)
}

# JSON verisini string'e dönüştürüp yazdırma
print(json.dumps(data))

{"right_angle": 0.08822044773719756, "left_angle": 0.11858101775956988, "cavity_depth": 2.0301277332356844, "roughness": 4.21311751638555}
